# Bloque 1: Stock

Primer Código Valida si todos tienen la misma Hoja

In [8]:
# import os
# import pandas as pd

# def validar_hoja_sheet1(carpeta):
#     archivos_sin_sheet1 = []

#     for root, _, files in os.walk(carpeta):
#         for file_name in files:
#             if file_name.lower().endswith((".xlsx")):
#                 try:
#                     excel_file_path = os.path.join(root, file_name)
#                     xls = pd.ExcelFile(excel_file_path)
#                     if 'Sheet1' not in xls.sheet_names:
#                         archivos_sin_sheet1.append(file_name)
#                 except Exception as e:
#                     print(f"Error al procesar el archivo {excel_file_path}: {e}")

#     return archivos_sin_sheet1

# # Obtener el directorio base del usuario
# carpeta_base = os.path.expanduser(r"~/DERCO CHILE REPUESTOS SpA/Planificación y abastecimiento - Documentos/KPI/Gerenciamiento Alcances/PBI/AFM/Stock")

# archivos_sin_sheet1 = validar_hoja_sheet1(carpeta_base)

# if archivos_sin_sheet1:
#     print("Archivos sin la hoja 'Sheet1' encontrados:")
#     for archivo in archivos_sin_sheet1:
#         print(archivo)
# else:
#     print("Todos los archivos tienen la hoja 'Sheet1'")


 Transformará todo los archivos de Stock que estén en la ruta (Folder_Path), guardará todos los transformados a la carpeta Transformación\Stock. Si el archivo ya existe no lo transformará

In [9]:
import os
import pandas as pd
def transformar_stock_tienda():
    folder_path = os.path.join(os.path.expanduser("~"), "DERCO CHILE REPUESTOS SpA", "Planificación y abastecimiento - Documentos", "KPI", "Gerenciamiento Alcances", "PBI", "AFM", "Stock")
    output_folder_path = os.path.join(os.path.expanduser("~"), "DERCO CHILE REPUESTOS SpA", "Planificación y abastecimiento - Documentos", "KPI", "Gerenciamiento Alcances", "PBI", "TR", "Stock")
    output_folder_path_2024 = os.path.join(output_folder_path, "2024")
    if not os.path.exists(output_folder_path):
        os.makedirs(output_folder_path)
    if not os.path.exists(output_folder_path_2024):
        os.makedirs(output_folder_path_2024)
    for root, _, files in os.walk(folder_path):
        for file_name in files:
            if file_name.endswith((".xlsx", ".XLSX")):
                excel_file_path = os.path.join(root, file_name)
                if '2024' in file_name:
                    output_path = output_folder_path_2024
                else:
                    output_path = output_folder_path
                parquet_file_name = file_name.replace('.xlsx', '_Sheet1.parquet').replace('.XLSX', '_Sheet1.parquet')
                parquet_file_path = os.path.join(output_path, parquet_file_name)
                if not os.path.exists(parquet_file_path):
                    try:
                        df = pd.read_excel(excel_file_path, sheet_name='Sheet1')
                        df.to_parquet(parquet_file_path, index=False)
                        print(f"Convertido {excel_file_path} a {parquet_file_path}")
                    except Exception as e:
                        print(f"Error al procesar el archivo {excel_file_path}: {e}")
transformar_stock_tienda()

Concatenar STOCK

In [10]:
import os
import pandas as pd
from datetime import datetime

def limpiar_y_procesar_archivos(base_folder_path, output_folder_path, folder_name):
    today = datetime.today().strftime('%Y-%m-%d')
    consolidado_folder_path = os.path.join(output_folder_path, "Consolidados", folder_name)
    
    if not os.path.exists(consolidado_folder_path):
        os.makedirs(consolidado_folder_path)

    all_files = []
    for root, _, files in os.walk(base_folder_path):
        for file_name in files:
            if file_name.lower().endswith((".parquet")):  # .lower() para manejar .PARQUET y .parquet
                all_files.append(os.path.join(root, file_name))

    if not all_files:
        print(f"No se encontraron archivos en {base_folder_path}.")
        return
    
    all_columns = [
        'Almacén', 'Centro', 'Inspecc.de calidad', 'Libre utilización', 'Material',
        'Moneda', 'SEM', 'Trans./Trasl.', 'Val.trans.c/cond.', 'Valor en insp.cal.',
        'Valor libre util.'
    ]

    dfs = []
    for file in all_files:
        try:
            df = pd.read_parquet(file)
            if 'Valoreninsp.cal.' in df.columns:
                df.rename(columns={'Valoreninsp.cal.': 'Valor en insp.cal.'}, inplace=True)
            for col in df.select_dtypes(include=['object']).columns:
                df[col] = df[col].str.strip()
            for col in ['Almacén', 'Centro', 'Material', 'Valor libre util.', 'Valor en insp.cal.', 'Val.trans.c/cond.']:
                if col in df.columns:
                    df[col] = pd.to_numeric(df[col], errors='coerce')
            df = df.reindex(columns=all_columns)
            df = df[pd.to_datetime(df['SEM'], errors='coerce') > '2023-09-18']
            dfs.append((file, df))
        except Exception as e:
            print(f"Error al leer el archivo {file}: {e}")

    if not dfs:
        print(f"No se pudieron leer archivos en {base_folder_path}.")
        return

    all_columns_set = set(all_columns)
    
    dfs_aligned = []
    for file, df in dfs:
        missing_columns = all_columns_set - set(df.columns)
        if missing_columns:
            print(f"El archivo {file} falta las columnas: {missing_columns}")
        extra_columns = set(df.columns) - all_columns_set
        if extra_columns:
            print(f"El archivo {file} tiene columnas adicionales: {extra_columns}")
        dfs_aligned.append(df)

    concatenated_df = pd.concat(dfs_aligned, ignore_index=True)
    output_file_name = f"{folder_name}_{today}.parquet"
    
    try:
        concatenated_df.to_parquet(os.path.join(consolidado_folder_path, output_file_name), index=False)
        print(f"Archivo Parquet guardado exitosamente: {output_file_name}")
    except Exception as e:
        print(f"Error al guardar el archivo Parquet {output_file_name}: {e}")

# Rutas ajustadas para el directorio del usuario
base_folder_path = os.path.expanduser(r"~/DERCO CHILE REPUESTOS SpA/Planificación y abastecimiento - Documentos/KPI/Gerenciamiento Alcances/PBI/TR/Stock")
output_folder_path = os.path.expanduser(r"~/DERCO CHILE REPUESTOS SpA/Planificación y abastecimiento - Documentos/KPI/Gerenciamiento Alcances/PBI/TR")

limpiar_y_procesar_archivos(base_folder_path, output_folder_path, "Stock")


Archivo Parquet guardado exitosamente: Stock_2024-11-15.parquet


# Bloque 2: Stock tienda
 Transformará todo los archivos de Stock Tienda que estén en la ruta (Folder_Path), guardará todos los transformados a la carpeta Transformación\Stock Tienda. Si el archivo ya existe no lo transformará


In [11]:
import os
import pandas as pd

def transformar_stock_tienda():
    # Rutas ajustadas para el directorio del usuario
    folder_path = os.path.expanduser(r"~/DERCO CHILE REPUESTOS SpA/Planificación y abastecimiento - Documentos/KPI/Gerenciamiento Alcances/PBI/AFM/Stock tienda")
    output_folder_path = os.path.expanduser(r"~/DERCO CHILE REPUESTOS SpA/Planificación y abastecimiento - Documentos/KPI/Gerenciamiento Alcances/PBI/TR/Stock tienda")
    output_folder_path_2024 = os.path.join(output_folder_path, "2024")

    # Crear carpetas si no existen
    if not os.path.exists(output_folder_path):
        os.makedirs(output_folder_path)

    if not os.path.exists(output_folder_path_2024):
        os.makedirs(output_folder_path_2024)

    # Iterar sobre los archivos en el directorio
    for root, _, files in os.walk(folder_path):
        for file_name in files:
            if file_name.lower().endswith(".xlsx"):  # Manejar .xlsx en minúsculas
                excel_file_path = os.path.join(root, file_name)

                # Determinar la ruta de salida según el nombre del archivo
                if '2024' in file_name:
                    output_path = output_folder_path_2024
                else:
                    output_path = output_folder_path

                # Generar el nombre del archivo parquet
                parquet_file_name = file_name.lower().replace('.xlsx', '_Sheet1.parquet')
                parquet_file_path = os.path.join(output_path, parquet_file_name)

                if os.path.exists(parquet_file_path):
                    print(f"El archivo {parquet_file_path} ya existe. Omitiendo TR.")
                    continue

                try:
                    print(f"Intentando leer el archivo: {excel_file_path}")
                    df = pd.read_excel(excel_file_path, sheet_name='Sheet1')
                    
                    # Limpiar espacios en blanco y convertir a float
                    df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
                    for column in df.select_dtypes(include=['object']).columns:
                        df[column] = pd.to_numeric(df[column].str.replace(' ', ''), errors='coerce')
                    
                    df.to_parquet(parquet_file_path, index=False)
                    print(f"Convertido {excel_file_path} a {parquet_file_path}")
                except Exception as e:
                    print(f"Error al procesar el archivo {excel_file_path}: {e}")

transformar_stock_tienda()


El archivo C:\Users\etorres.DERCOPARTS/DERCO CHILE REPUESTOS SpA/Planificación y abastecimiento - Documentos/KPI/Gerenciamiento Alcances/PBI/TR/Stock tienda\2024\stock tiendas sem1 2024_Sheet1.parquet ya existe. Omitiendo TR.
El archivo C:\Users\etorres.DERCOPARTS/DERCO CHILE REPUESTOS SpA/Planificación y abastecimiento - Documentos/KPI/Gerenciamiento Alcances/PBI/TR/Stock tienda\2024\stock tiendas sem10 2024_Sheet1.parquet ya existe. Omitiendo TR.
El archivo C:\Users\etorres.DERCOPARTS/DERCO CHILE REPUESTOS SpA/Planificación y abastecimiento - Documentos/KPI/Gerenciamiento Alcances/PBI/TR/Stock tienda\2024\stock tiendas sem11 2024_Sheet1.parquet ya existe. Omitiendo TR.
El archivo C:\Users\etorres.DERCOPARTS/DERCO CHILE REPUESTOS SpA/Planificación y abastecimiento - Documentos/KPI/Gerenciamiento Alcances/PBI/TR/Stock tienda\2024\stock tiendas sem12 2024_Sheet1.parquet ya existe. Omitiendo TR.
El archivo C:\Users\etorres.DERCOPARTS/DERCO CHILE REPUESTOS SpA/Planificación y abastecimien

In [12]:
import os
import pandas as pd
from datetime import datetime

def limpiar_y_procesar_archivos(base_folder_path, output_folder_path, folder_name):
    today = datetime.today().strftime('%Y-%m-%d')
    consolidado_folder_path = os.path.join(output_folder_path, "Stock_tienda", "sin_centro")
    
    if not os.path.exists(consolidado_folder_path):
        os.makedirs(consolidado_folder_path)

    all_files = []
    
    for root, _, files in os.walk(base_folder_path):
        for file_name in files:
            if file_name.lower().endswith(".parquet"):  # Manejar .parquet en minúsculas
                all_files.append(os.path.join(root, file_name))

    if not all_files:
        print(f"No se encontraron archivos en {base_folder_path}.")
        return
    
    dfs = []
    for file in all_files:
        try:
            df = pd.read_parquet(file)
            for col in df.columns:
                if col != 'SEM':
                    df[col] = df[col].astype(str).str.strip()
            print(f"Archivo {file} leído y limpiado.")
            dfs.append(df)
        except Exception as e:
            print(f"Error al leer el archivo {file}: {e}")

    if not dfs:
        print(f"No se pudieron leer archivos en {base_folder_path}.")
        return

    all_columns = [set(df.columns) for df in dfs]
    common_columns = set.intersection(*all_columns)

    if not common_columns:
        print("No hay columnas comunes entre los archivos. Abortando la concatenación.")
        return
    
    concatenated_df = pd.concat(dfs, ignore_index=True)
    print("Archivos concatenados.")

    concatenated_df['SEM'] = pd.to_datetime(concatenated_df['SEM'], errors='coerce')
    concatenated_df = concatenated_df[concatenated_df['SEM'] > '2023-09-18']
    print("Filtrado de registros realizado con éxito en la columna 'SEM'.")

    # Eliminar la columna 'Almacén' si existe
    if 'Almacén' in concatenated_df.columns:
        concatenated_df.drop(columns=['Almacén'], inplace=True)
        print("Columna 'Almacén' eliminada.")

    # Si existe la columna 'Centro', asignarle el valor 0
    if 'Centro' in concatenated_df.columns:
        concatenated_df['Centro'] = 0
        print("Columna 'Centro' encontrada y asignada con valor 0.")

    # Convertir las columnas especificadas a numéricas
    cols_to_numeric = ['Libre utilización', 'Valor libre util.', 'Trans./Trasl.', 
                       'Val.trans.c/cond.', 'Inspecc.de calidad', 'Valor en insp.cal.']
    for col in cols_to_numeric:
        if col in concatenated_df.columns:
            concatenated_df[col] = pd.to_numeric(concatenated_df[col], errors='coerce')
            print(f"Columna '{col}' convertida a numérica.")

    # Eliminar la columna 'Centro' si existe
    if 'Centro' in concatenated_df.columns:
        concatenated_df.drop(columns=['Centro'], inplace=True)
        print("Columna 'Centro' eliminada.")

    # Agrupar los datos por 'Material' y 'SEM', y sumar las columnas numéricas
    df_agrupado = concatenated_df.groupby(['Material', 'SEM']).sum(min_count=1).reset_index()
    print("Datos agrupados por 'Material' y 'SEM'.")
    
    print(df_agrupado.head())
    
    # Guardar el DataFrame final con el nombre que incluye '_SinCentro'
    output_file_name = f"{folder_name}_{today}_SinCentro.parquet"
    
    try:
        df_agrupado.to_parquet(os.path.join(consolidado_folder_path, output_file_name), index=False, compression='snappy')
        print(f"Archivo Parquet guardado exitosamente en: {os.path.join(consolidado_folder_path, output_file_name)}")
    except Exception as e:
        print(f"Error al guardar el archivo Parquet {output_file_name}: {e}")

# Rutas ajustadas para el directorio del usuario
base_folder_path = os.path.expanduser(r"~/DERCO CHILE REPUESTOS SpA/Planificación y abastecimiento - Documentos/KPI/Gerenciamiento Alcances/PBI/TR/Stock tienda/2024")
output_folder_path = os.path.expanduser(r"~\DERCO CHILE REPUESTOS SpA\Planificación y abastecimiento - Documentos\KPI\Gerenciamiento Alcances\PBI")

limpiar_y_procesar_archivos(base_folder_path, output_folder_path, "Stock_tienda")


Archivo C:\Users\etorres.DERCOPARTS/DERCO CHILE REPUESTOS SpA/Planificación y abastecimiento - Documentos/KPI/Gerenciamiento Alcances/PBI/TR/Stock tienda/2024\Stock Tiendas Sem1 2024_Sheet1.parquet leído y limpiado.
Archivo C:\Users\etorres.DERCOPARTS/DERCO CHILE REPUESTOS SpA/Planificación y abastecimiento - Documentos/KPI/Gerenciamiento Alcances/PBI/TR/Stock tienda/2024\Stock Tiendas Sem10 2024_Sheet1.parquet leído y limpiado.
Archivo C:\Users\etorres.DERCOPARTS/DERCO CHILE REPUESTOS SpA/Planificación y abastecimiento - Documentos/KPI/Gerenciamiento Alcances/PBI/TR/Stock tienda/2024\Stock Tiendas Sem11 2024_Sheet1.parquet leído y limpiado.
Archivo C:\Users\etorres.DERCOPARTS/DERCO CHILE REPUESTOS SpA/Planificación y abastecimiento - Documentos/KPI/Gerenciamiento Alcances/PBI/TR/Stock tienda/2024\Stock Tiendas Sem12 2024_Sheet1.parquet leído y limpiado.
Archivo C:\Users\etorres.DERCOPARTS/DERCO CHILE REPUESTOS SpA/Planificación y abastecimiento - Documentos/KPI/Gerenciamiento Alcances

# Bloque 3: Transito# 

Transformar y guardar transito

In [13]:
import os
import pandas as pd

def transformar_stock_tienda():
    # Rutas ajustadas para el directorio del usuario
    folder_path = os.path.expanduser(r"~/DERCO CHILE REPUESTOS SpA/Planificación y abastecimiento - Documentos/KPI/Gerenciamiento Alcances/PBI/AFM/Transito")
    output_folder_path = os.path.expanduser(r"~/DERCO CHILE REPUESTOS SpA/Planificación y abastecimiento - Documentos/KPI/Gerenciamiento Alcances/PBI/TR/Transito")
    output_folder_path_2024 = os.path.join(output_folder_path, "2024")

    # Crear carpetas si no existen
    os.makedirs(output_folder_path, exist_ok=True)
    os.makedirs(output_folder_path_2024, exist_ok=True)

    # Iterar sobre los archivos en el directorio
    for root, _, files in os.walk(folder_path):
        for file_name in files:
            if file_name.lower().endswith(".xlsx"):  # Manejar tanto .xlsx como .XLSX
                excel_file_path = os.path.join(root, file_name)

                # Determinar la ruta de salida según el nombre del archivo
                if '2024' in file_name:
                    output_path = output_folder_path_2024
                else:
                    output_path = output_folder_path

                # Generar el nombre del archivo parquet
                parquet_file_name = file_name.lower().replace('.xlsx', '_Sheet1.parquet')
                parquet_file_path = os.path.join(output_path, parquet_file_name)

                # Verificar si el archivo Parquet ya existe
                if os.path.exists(parquet_file_path):
                    print(f"El archivo {parquet_file_path} ya existe. Omitiendo transformación.")
                    continue

                try:
                    # Leer y transformar el archivo Excel
                    df = pd.read_excel(excel_file_path, sheet_name='Sheet1')
                    df = df.astype(str)  # Convertir todos los datos a cadena de texto
                    df.to_parquet(parquet_file_path, index=False)
                    print(f"Convertido {excel_file_path} a {parquet_file_path}")
                except Exception as e:
                    print(f"Error al procesar el archivo {excel_file_path}: {e}")

transformar_stock_tienda()


El archivo C:\Users\etorres.DERCOPARTS/DERCO CHILE REPUESTOS SpA/Planificación y abastecimiento - Documentos/KPI/Gerenciamiento Alcances/PBI/TR/Transito\2024\tr semana 1 consolidado 2024_Sheet1.parquet ya existe. Omitiendo transformación.
El archivo C:\Users\etorres.DERCOPARTS/DERCO CHILE REPUESTOS SpA/Planificación y abastecimiento - Documentos/KPI/Gerenciamiento Alcances/PBI/TR/Transito\2024\tr semana 10 consolidado 2024_Sheet1.parquet ya existe. Omitiendo transformación.
El archivo C:\Users\etorres.DERCOPARTS/DERCO CHILE REPUESTOS SpA/Planificación y abastecimiento - Documentos/KPI/Gerenciamiento Alcances/PBI/TR/Transito\2024\tr semana 11 consolidado 2024_Sheet1.parquet ya existe. Omitiendo transformación.
El archivo C:\Users\etorres.DERCOPARTS/DERCO CHILE REPUESTOS SpA/Planificación y abastecimiento - Documentos/KPI/Gerenciamiento Alcances/PBI/TR/Transito\2024\tr semana 12 consolidado 2024_Sheet1.parquet ya existe. Omitiendo transformación.
El archivo C:\Users\etorres.DERCOPARTS/DE

detectar columnas unicas

In [14]:
import os
import pandas as pd

def listar_columnas_unicas(base_folder_path):
    all_files = []
    columnas_unicas = set()
    
    for root, _, files in os.walk(base_folder_path):
        for file_name in files:
            if file_name.endswith((".parquet", ".PARQUET")):
                all_files.append(os.path.join(root, file_name))

    if not all_files:
        print(f"No se encontraron archivos en {base_folder_path}.")
        return
    
    for file in all_files:
        try:
            df = pd.read_parquet(file)
            columnas_unicas.update(df.columns)
        except Exception as e:
            print(f"Error al leer el archivo {file}: {e}")

    print("Columnas únicas en todos los archivos de Transito:")
    for col in sorted(columnas_unicas):
        print(col)

base_folder_path = os.path.expanduser(r"~/DERCO CHILE REPUESTOS SpA/Planificación y abastecimiento - Documentos/KPI/Gerenciamiento Alcances/PBI/TR/Transito")
listar_columnas_unicas(base_folder_path)


Columnas únicas en todos los archivos de Transito:
Cantidad
Centro
Clase doc.
Descripcion
Fecha
Grupo de Compra
Material
Origen
Proveedor
SEM
Sector
Status TR Fact
Tipo


Concatenar transito

In [15]:
import os
import pandas as pd
from datetime import datetime

def procesar_archivos(base_folder_path, output_folder_path, folder_name):
    today = datetime.today().strftime('%Y-%m-%d')
    consolidado_folder_path = os.path.join(output_folder_path, "Consolidados", folder_name)
    
    if not os.path.exists(consolidado_folder_path):
        os.makedirs(consolidado_folder_path)

    all_files = []
    
    for root, _, files in os.walk(base_folder_path):
        for file_name in files:
            if file_name.endswith((".parquet", ".PARQUET")):
                all_files.append(os.path.join(root, file_name))

    if not all_files:
        print(f"No se encontraron archivos en {base_folder_path}.")
        return
    
    # Columnas únicas basadas en el listado proporcionado y ordenadas alfabéticamente
    all_columns = [
        'Cantidad', 'Centro', 'Clase doc.', 'Descripcion', 'Fecha', 'Grupo de Compra',
        'Material', 'Origen', 'Proveedor', 'SEM', 'Sector', 'Status TR Fact', 'Tipo'
    ]

    dfs = []
    for file in all_files:
        try:
            df = pd.read_parquet(file)
            # Convertir todas las columnas a string excepto 'SEM' y 'Fecha'
            for col in df.columns:
                if col not in ['SEM', 'Fecha']:
                    df[col] = df[col].astype(str).str.strip()
            # Truncar 'Fecha' y 'SEM' a sus primeros 10 caracteres
            if 'Fecha' in df.columns:
                df['Fecha'] = df['Fecha'].astype(str).str[:10]
                df['Fecha'] = pd.to_datetime(df['Fecha'], errors='coerce')
            if 'SEM' in df.columns:
                df['SEM'] = df['SEM'].astype(str).str[:10]
            # Alinear las columnas con las columnas únicas
            df = df.reindex(columns=all_columns)
            # Convertir columnas numéricas que podrían estar en formato string
            for col in ['Cantidad', 'Material', 'Centro']:
                if col in df.columns:
                    df[col] = pd.to_numeric(df[col], errors='coerce')
            dfs.append(df)
        except Exception as e:
            print(f"Error al leer el archivo {file}: {e}")

    if not dfs:
        print(f"No se pudieron leer archivos en {base_folder_path}.")
        return

    concatenated_df = pd.concat(dfs, ignore_index=True)
    output_file_name = f"{folder_name}_{today}.parquet"
    
    try:
        concatenated_df.to_parquet(os.path.join(consolidado_folder_path, output_file_name), index=False)
        print(f"Archivo Parquet guardado exitosamente: {output_file_name}")
    except Exception as e:
        print(f"Error al guardar el archivo Parquet {output_file_name}: {e}")

base_folder_path = os.path.expanduser(r"~/DERCO CHILE REPUESTOS SpA/Planificación y abastecimiento - Documentos/KPI/Gerenciamiento Alcances/PBI/TR/Transito")
output_folder_path = os.path.expanduser(r"~/DERCO CHILE REPUESTOS SpA/Planificación y abastecimiento - Documentos/KPI/Gerenciamiento Alcances/PBI/TR")
procesar_archivos(base_folder_path, output_folder_path, "Transito")


Archivo Parquet guardado exitosamente: Transito_2024-11-15.parquet


# Bloque 4: Venta promedio

In [16]:
import os
import pandas as pd

def transformar_stock_tienda():
    folder_path = os.path.expanduser(r"~/DERCO CHILE REPUESTOS SpA/Planificación y abastecimiento - Documentos/KPI/Gerenciamiento Alcances/PBI/AFM/Venta promedio")
    output_folder_path = os.path.expanduser(r"~/DERCO CHILE REPUESTOS SpA/Planificación y abastecimiento - Documentos/KPI/Gerenciamiento Alcances/PBI/TR/Venta promedio")
    output_folder_path_2024 = os.path.join(output_folder_path, "2024")

    if not os.path.exists(output_folder_path):
        os.makedirs(output_folder_path)

    if not os.path.exists(output_folder_path_2024):
        os.makedirs(output_folder_path_2024)

    for root, _, files in os.walk(folder_path):
        for file_name in files:
            if file_name.endswith((".xlsx", ".XLSX")):
                excel_file_path = os.path.join(root, file_name)

                if '2024' in file_name:
                    output_path = output_folder_path_2024
                else:
                    output_path = output_folder_path

                parquet_file_name = file_name.replace('.xlsx', '_Sheet1.parquet').replace('.XLSX', '_Sheet1.parquet')
                parquet_file_path = os.path.join(output_path, parquet_file_name)

                if os.path.exists(parquet_file_path):
                    print(f"El archivo {parquet_file_path} ya existe. Omitiendo transformación.")
                    continue

                try:
                    print(f"Intentando leer el archivo: {excel_file_path}")
                    df = pd.read_excel(excel_file_path, sheet_name='Hoja1')
                    df.to_parquet(parquet_file_path, index=False)
                    print(f"Convertido {excel_file_path} a {parquet_file_path}")
                except Exception as e:
                    print(f"Error al procesar el archivo {excel_file_path}: {e}")

transformar_stock_tienda()


El archivo C:\Users\etorres.DERCOPARTS/DERCO CHILE REPUESTOS SpA/Planificación y abastecimiento - Documentos/KPI/Gerenciamiento Alcances/PBI/TR/Venta promedio\2024\Venta Abril 2024_Sheet1.parquet ya existe. Omitiendo transformación.
El archivo C:\Users\etorres.DERCOPARTS/DERCO CHILE REPUESTOS SpA/Planificación y abastecimiento - Documentos/KPI/Gerenciamiento Alcances/PBI/TR/Venta promedio\2024\Venta Agosto 2024_Sheet1.parquet ya existe. Omitiendo transformación.
El archivo C:\Users\etorres.DERCOPARTS/DERCO CHILE REPUESTOS SpA/Planificación y abastecimiento - Documentos/KPI/Gerenciamiento Alcances/PBI/TR/Venta promedio\2024\Venta Enero 2024_Sheet1.parquet ya existe. Omitiendo transformación.
El archivo C:\Users\etorres.DERCOPARTS/DERCO CHILE REPUESTOS SpA/Planificación y abastecimiento - Documentos/KPI/Gerenciamiento Alcances/PBI/TR/Venta promedio\2024\Venta Febrero 2024_Sheet1.parquet ya existe. Omitiendo transformación.
El archivo C:\Users\etorres.DERCOPARTS/DERCO CHILE REPUESTOS SpA/

In [17]:
import os
import pandas as pd

def listar_columnas_unicas(base_folder_path):
    all_files = []
    columnas_unicas = set()
    
    for root, _, files in os.walk(base_folder_path):
        for file_name in files:
            if file_name.endswith((".parquet", ".PARQUET")):
                all_files.append(os.path.join(root, file_name))

    if not all_files:
        print(f"No se encontraron archivos en {base_folder_path}.")
        return
    
    for file in all_files:
        try:
            df = pd.read_parquet(file)
            columnas_unicas.update(df.columns)
        except Exception as e:
            print(f"Error al leer el archivo {file}: {e}")

    print("Columnas únicas en todos los archivos de Ventas:")
    for col in sorted(columnas_unicas):
        print(col)

base_folder_path = os.path.expanduser(r"~/DERCO CHILE REPUESTOS SpA/Planificación y abastecimiento - Documentos/KPI/Gerenciamiento Alcances/PBI/TR/Venta promedio")
listar_columnas_unicas(base_folder_path)


Columnas únicas en todos los archivos de Ventas:
Material
Unnamed: 3
Vta promedio
fecha


CONCATENAR VTA PROMEDIO

In [18]:
import os
import pandas as pd
from datetime import datetime

def procesar_archivos(base_folder_path, output_folder_path, folder_name):
    today = datetime.today().strftime('%Y-%m-%d')
    consolidado_folder_path = os.path.join(output_folder_path, "Consolidados", folder_name)
    
    if not os.path.exists(consolidado_folder_path):
        os.makedirs(consolidado_folder_path)

    all_files = []
    
    for root, _, files in os.walk(base_folder_path):
        for file_name in files:
            if file_name.endswith((".parquet", ".PARQUET")):
                all_files.append(os.path.join(root, file_name))

    if not all_files:
        print(f"No se encontraron archivos en {base_folder_path}.")
        return
    
    required_columns = {'Material', 'Vta promedio', 'fecha'}
    dfs = []
    archivos_con_error = []

    for file in all_files:
        try:
            df = pd.read_parquet(file)
            df['Material'] = df['Material'].astype(str)
            # Verificar si las columnas requeridas están en el DataFrame
            if not required_columns.issubset(df.columns):
                archivos_con_error.append(file)
                print(f"El archivo {file} no tiene las columnas requeridas: {required_columns - set(df.columns)}")
                continue
            # Seleccionar solo las columnas necesarias
            df = df[list(required_columns)]
            dfs.append(df)
        except Exception as e:
            print(f"Error al leer el archivo {file}: {e}")

    if not dfs:
        print(f"No se pudieron leer archivos en {base_folder_path}.")
        return

    concatenated_df = pd.concat(dfs, ignore_index=True)
    output_file_name = f"{folder_name}_{today}.parquet"
    
    try:
        concatenated_df.to_parquet(os.path.join(consolidado_folder_path, output_file_name), index=False)
        print(f"Archivo Parquet guardado exitosamente: {output_file_name}")
    except Exception as e:
        print(f"Error al guardar el archivo Parquet {output_file_name}: {e}")

    if archivos_con_error:
        print(f"Archivos con errores: {archivos_con_error}")

base_folder_path = os.path.expanduser(r"~/DERCO CHILE REPUESTOS SpA/Planificación y abastecimiento - Documentos/KPI/Gerenciamiento Alcances/PBI/TR/Venta promedio")
output_folder_path = os.path.expanduser(r"~/DERCO CHILE REPUESTOS SpA/Planificación y abastecimiento - Documentos/KPI/Gerenciamiento Alcances/PBI/TR")
procesar_archivos(base_folder_path, output_folder_path, "Venta_promedio")


Archivo Parquet guardado exitosamente: Venta_promedio_2024-11-15.parquet


# Bloque 5: FC Promedio

In [19]:
import os
import pandas as pd

def transformar_stock_tienda():
    folder_path = os.path.expanduser(r"~/DERCO CHILE REPUESTOS SpA/Planificación y abastecimiento - Documentos/KPI/Gerenciamiento Alcances/PBI/AFM/FC Promedio")
    output_folder_path = os.path.expanduser(r"~/DERCO CHILE REPUESTOS SpA/Planificación y abastecimiento - Documentos/KPI/Gerenciamiento Alcances/PBI/TR/FC Promedio")
    output_folder_path_2024 = os.path.join(output_folder_path, "2024")

    if not os.path.exists(output_folder_path):
        os.makedirs(output_folder_path)

    if not os.path.exists(output_folder_path_2024):
        os.makedirs(output_folder_path_2024)

    for root, _, files in os.walk(folder_path):
        for file_name in files:
            if file_name.endswith((".xlsx", ".XLSX")):
                excel_file_path = os.path.join(root, file_name)

                if '2024' in file_name:
                    output_path = output_folder_path_2024
                else:
                    output_path = output_folder_path

                parquet_file_name = file_name.replace('.xlsx', '_Sheet1.parquet').replace('.XLSX', '_Sheet1.parquet')
                parquet_file_path = os.path.join(output_path, parquet_file_name)

                if os.path.exists(parquet_file_path):
                    print(f"El archivo {parquet_file_path} ya existe. Omitiendo transformación.")
                    continue

                try:
                    print(f"Intentando leer el archivo: {excel_file_path}")
                    df = pd.read_excel(excel_file_path, sheet_name='BASE')
                    df.to_parquet(parquet_file_path, index=False)
                    print(f"Convertido {excel_file_path} a {parquet_file_path}")
                except Exception as e:
                    print(f"Error al procesar el archivo {excel_file_path}: {e}")

transformar_stock_tienda()


El archivo C:\Users\etorres.DERCOPARTS/DERCO CHILE REPUESTOS SpA/Planificación y abastecimiento - Documentos/KPI/Gerenciamiento Alcances/PBI/TR/FC Promedio\2024\FC Abril 2024_Sheet1.parquet ya existe. Omitiendo transformación.
El archivo C:\Users\etorres.DERCOPARTS/DERCO CHILE REPUESTOS SpA/Planificación y abastecimiento - Documentos/KPI/Gerenciamiento Alcances/PBI/TR/FC Promedio\2024\FC Agosto 2024_Sheet1.parquet ya existe. Omitiendo transformación.
El archivo C:\Users\etorres.DERCOPARTS/DERCO CHILE REPUESTOS SpA/Planificación y abastecimiento - Documentos/KPI/Gerenciamiento Alcances/PBI/TR/FC Promedio\2024\FC Enero 2024_Sheet1.parquet ya existe. Omitiendo transformación.
El archivo C:\Users\etorres.DERCOPARTS/DERCO CHILE REPUESTOS SpA/Planificación y abastecimiento - Documentos/KPI/Gerenciamiento Alcances/PBI/TR/FC Promedio\2024\FC Febrero 2024_Sheet1.parquet ya existe. Omitiendo transformación.
El archivo C:\Users\etorres.DERCOPARTS/DERCO CHILE REPUESTOS SpA/Planificación y abasteci

In [20]:
import os
import pandas as pd
from datetime import datetime

def procesar_archivos(base_folder_path, output_folder_path, folder_name):
    today = datetime.today().strftime('%Y-%m-%d')
    consolidado_folder_path = os.path.join(output_folder_path, "Consolidados", folder_name)
    
    if not os.path.exists(consolidado_folder_path):
        os.makedirs(consolidado_folder_path)

    all_files = []
    
    for root, _, files in os.walk(base_folder_path):
        for file_name in files:
            if file_name.endswith((".parquet", ".PARQUET")):
                all_files.append(os.path.join(root, file_name))

    if not all_files:
        print(f"No se encontraron archivos en {base_folder_path}.")
        return
    
    dfs = []
    for file in all_files:
        try:
            print(f"Intentando leer el archivo: {file}")
            df = pd.read_parquet(file)
            print(f"Archivo leído exitosamente: {file}")
            dfs.append(df)
        except Exception as e:
            print(f"Error al leer el archivo {file}: {e}")

    if not dfs:
        print(f"No se pudieron leer archivos en {base_folder_path}.")
        return

    # Comprobación de columnas
    all_columns = [set(df.columns) for df in dfs]
    common_columns = set.intersection(*all_columns)

    if not common_columns:
        print("No hay columnas comunes entre los archivos. Abortando la concatenación.")
        return
    
    for df in dfs:
        if set(df.columns) != common_columns:
            print(f"Las columnas del archivo no coinciden con las columnas comunes: {df.columns}")
    
    concatenated_df = pd.concat(dfs, ignore_index=True)
    output_file_name = f"{folder_name}_{today}.parquet"
    
    try:
        concatenated_df.to_parquet(os.path.join(consolidado_folder_path, output_file_name), index=False)
        print(f"Archivo Parquet guardado exitosamente: {output_file_name}")
    except Exception as e:
        print(f"Error al guardar el archivo Parquet {output_file_name}: {e}")

base_folder_path = os.path.expanduser(r"~/DERCO CHILE REPUESTOS SpA/Planificación y abastecimiento - Documentos/KPI/Gerenciamiento Alcances/PBI/TR/FC Promedio")
output_folder_path = os.path.expanduser(r"~/DERCO CHILE REPUESTOS SpA/Planificación y abastecimiento - Documentos/KPI/Gerenciamiento Alcances/PBI/TR")
procesar_archivos(base_folder_path, output_folder_path, "FC_Promedio")


Intentando leer el archivo: C:\Users\etorres.DERCOPARTS/DERCO CHILE REPUESTOS SpA/Planificación y abastecimiento - Documentos/KPI/Gerenciamiento Alcances/PBI/TR/FC Promedio\FC Abril 2023_Sheet1.parquet
Archivo leído exitosamente: C:\Users\etorres.DERCOPARTS/DERCO CHILE REPUESTOS SpA/Planificación y abastecimiento - Documentos/KPI/Gerenciamiento Alcances/PBI/TR/FC Promedio\FC Abril 2023_Sheet1.parquet
Intentando leer el archivo: C:\Users\etorres.DERCOPARTS/DERCO CHILE REPUESTOS SpA/Planificación y abastecimiento - Documentos/KPI/Gerenciamiento Alcances/PBI/TR/FC Promedio\FC Agosto 2023_Sheet1.parquet
Archivo leído exitosamente: C:\Users\etorres.DERCOPARTS/DERCO CHILE REPUESTOS SpA/Planificación y abastecimiento - Documentos/KPI/Gerenciamiento Alcances/PBI/TR/FC Promedio\FC Agosto 2023_Sheet1.parquet
Intentando leer el archivo: C:\Users\etorres.DERCOPARTS/DERCO CHILE REPUESTOS SpA/Planificación y abastecimiento - Documentos/KPI/Gerenciamiento Alcances/PBI/TR/FC Promedio\FC Diciembre 2023

CONCATENAR FC

In [21]:
import os
import pandas as pd
from datetime import datetime

def procesar_archivos(base_folder_path, output_folder_path, folder_name):
    today = datetime.today().strftime('%Y-%m-%d')
    consolidado_folder_path = os.path.join(output_folder_path, "Consolidados", folder_name)
    
    if not os.path.exists(consolidado_folder_path):
        os.makedirs(consolidado_folder_path)

    all_files = []
    
    for root, _, files in os.walk(base_folder_path):
        for file_name in files:
            if file_name.endswith((".parquet", ".PARQUET")):
                all_files.append(os.path.join(root, file_name))

    if not all_files:
        print(f"No se encontraron archivos en {base_folder_path}.")
        return
    
    dfs = []
    for file in all_files:
        try:
            df = pd.read_parquet(file)
            dfs.append(df)
        except Exception as e:
            print(f"Error al leer el archivo {file}: {e}")

    if not dfs:
        print(f"No se pudieron leer archivos en {base_folder_path}.")
        return

    all_columns = [set(df.columns) for df in dfs]
    common_columns = set.intersection(*all_columns)

    if not common_columns:
        print("No hay columnas comunes entre los archivos. Abortando la concatenación.")
        return
    
    for df in dfs:
        if set(df.columns) != common_columns:
            print(f"Las columnas del archivo no coinciden con las columnas comunes: {df.columns}")
    
    concatenated_df = pd.concat(dfs, ignore_index=True)
    output_file_name = f"{folder_name}_{today}.parquet"
    
    try:
        concatenated_df.to_parquet(os.path.join(consolidado_folder_path, output_file_name), index=False)
        print(f"Archivo Parquet guardado exitosamente: {output_file_name}")
    except Exception as e:
        print(f"Error al guardar el archivo Parquet {output_file_name}: {e}")

base_folder_path = os.path.expanduser(r"~/DERCO CHILE REPUESTOS SpA/Planificación y abastecimiento - Documentos/KPI/Gerenciamiento Alcances/PBI/TR/FC Promedio")
output_folder_path = os.path.expanduser(r"~/DERCO CHILE REPUESTOS SpA/Planificación y abastecimiento - Documentos/KPI/Gerenciamiento Alcances/PBI/TR")
procesar_archivos(base_folder_path, output_folder_path, "FC_Promedio")


Archivo Parquet guardado exitosamente: FC_Promedio_2024-11-15.parquet
